In [76]:
% load_ext autoreload
% aimport utils
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from utils import *
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import plotly.express as px

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
%autoreload 1
deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv',
           error_bad_lines=False)
cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv',
                   error_bad_lines=False)
#cases['FIPS'] = cases['FIPS'].astype('int')
day_cases = get_day_cases()
counties = pd.read_csv('https://raw.githubusercontent.com/Yu-Group/covid19-severity-prediction/master/data/county_data_abridged.csv',
                      error_bad_lines=False)
county_cases = pd.read_csv('https://raw.githubusercontent.com/Yu-Group/covid19-severity-prediction/master/data/county_level/processed/nytimes_infections/nytimes_infections.csv',
                          error_bad_lines=False)

death_keep = [col+'_deaths' if (col in cases.columns and col != 'FIPS') else col for col in deaths.columns]
cases_keep = [col+'_cases' for col in cases.columns[12:]]
cases_and_deaths = cases.merge(deaths, on='FIPS', suffixes=('_cases', '_deaths'))[death_keep+cases_keep]
renamer = lambda name: name if (type(name) != str or '/' in name or '_deaths' not in name) else name[:-7]
cases_and_deaths = cases_and_deaths.rename(columns=renamer)
cases_and_deaths.head()

valid_territories = cases_and_deaths[(cases_and_deaths['Lat'] > 0) & ~np.isnan(cases_and_deaths['FIPS'])]
valid_territories.loc[:, 'FIPS'] = valid_territories['FIPS'].astype('int')
valid_counties = counties[~counties['countyFIPS'].isin(['City1', 'City2'])]
valid_counties.loc[:, 'countyFIPS'] = valid_counties['countyFIPS'].astype('int')

county_all = valid_counties.merge(valid_territories, left_on='countyFIPS', right_on='FIPS')
county_all = county_all.drop(columns=['Lat', 'Long_', 'Province_State', 'Country_Region', 'FIPS'])

C:\Users\2000s\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [43]:
[day_cases[i].shape for i in range(len(day_cases))]

[(50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18),
 (50, 18)]

### Deciding the Day to Split on



In [68]:
mort_rates = np.zeros((50, len(day_cases)))
for terr in range(50):
    mort_rates[terr, :] = get_trait_ts(terr, 'Mortality_Rate')

In [82]:
# Do some PCA to see if there are distinct clusters of behavior
# Apparently there are not.
mort_rates_centered = mort_rates - mort_rates.mean(axis=0)
pca = PCA(n_components=2)
pairs = pca.fit_transform(mort_rates_centered)
px.scatter(pairs[:, 0], pairs[:, 1], hover_name=day_cases[0]['Province_State'])

### Choosing a day to split on

We progressively step through the days and test them to see which has the highest variance in mortality rates. We find that the 30th day in the set of dataframes, corresponding to 

In [85]:
# Choosing which day to split on: take the day with highest variance in mora
max_var_day = np.argmax([np.std(mort_rates[:, i]) for i in range(mort_rates.shape[1])])
max_var_day

30

array([0.93976454, 0.04416661])